# Structured Output 
## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
from utils import get_completion, get_completion_with_structured_output, AZURE_OPENAI_API_GPT_35_MODEL, AZURE_OPENAI_API_GPT_4o_MODEL, extract_llm_response

model = AZURE_OPENAI_API_GPT_4o_MODEL

---

## Lesson

**LLM can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to LLM. It turns out, you can also ask LLM to **use XML tags to make its output clearer and more easily understandable** to humans.

### 1. Having the output in XML tags allows the end user to reliably get the output by writing a short program to extract the content between XML tags.

In [2]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a poetry about {ANIMAL}. Put it in <poetry> tags."

# Prefill for LLM's response
PREFILL = "<poetry>"

# Print LLM's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- LLM's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL, model=model))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a poetry about Cat. Put it in <poetry> tags.

ASSISTANT TURN:
<poetry>

------------------------------------- LLM's response -------------------------------------
==================LLM gpt-4o response======================
<poetry>  
Beneath the moon's soft silver glow,  
A shadow dances, sleek and slow.  
With eyes like amber, sharp and bright,  
The cat becomes the queen of night.  

Her velvet paws, so light, so fleet,  
Whisper secrets to the street.  
A hunter's grace, a mystic's air,  
She moves as though she isn't there.  

Her purr, a song of warmth and peace,  
A melody that bids time cease.  
Yet in her gaze, a wild spark,  
A fragment of the untamed dark.  

She leaps, she prowls, she claims her throne,  
A creature both of flesh and stone.  
A guardian of dreams and lore,  
A spirit ancient, evermore.  

Oh, feline muse, so fierce, so free,  
A living riddle

In [ ]:
PROMPT = """ 
Write a simple python function that
 1. Ask me for a number in mile
 2. It converts miles to kilometers
"""
PREFILL = "import"

res = get_completion(PROMPT, prefill=PREFILL, model=model)
print("\n------------------------------------- LLM's response -------------------------------------")
print(res)

extracted_res = extract_llm_response(res, tag="python")
print("\n------------------------------------- Extracted LLM's response -------------------------------------")
print(extracted_res)


==================LLM gpt-4o-08-06 response======================

------------------------------------- LLM's response -------------------------------------
```python
def miles_to_kilometers():
    try:
        # Ask the user for a number in miles
        miles = float(input("Enter a distance in miles: "))
        
        # Conversion factor from miles to kilometers
        conversion_factor = 1.60934
        
        # Convert miles to kilometers
        kilometers = miles * conversion_factor
        
        # Display the result
        print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")
    except ValueError:
        print("Please enter a valid number.")

# Call the function
miles_to_kilometers()
```

This function prompts the user to enter a distance in miles, converts it to kilometers using the conversion factor (1 mile = 1.60934 kilometers), and then prints the result. It also includes error handling to ensure that the input is a valid number.

--------------------

LLM also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill LLM's response with the opening bracket, `{`}.

### 2. Using Few Shots to get a JSON output

For LLMs or SLMs that do not support JSON-mode or function calling, you can use few shots to get a JSON output.

In [ ]:
SYSTEM_PROMPT = """You are an AI assistant specialized in parsing exception logs and extracting key information into a structured JSON format.

Given the logs (wrapped in ###) below create a JSON object with keys ""ExceptionType"",  ""Message"" and ""StackTrace"".

For the ""ExceptionType"" property,
1. Only assign who raised the exception, e.g. (""Type 'SetUpdateURI' of Role 'URP' raised an exception""), not the stack trace and other details.
2. You can find the exception type before the stack trace.
4. The value of ""ExceptionType"" should be extracted directly from the logs and not be modified.

For the ""Message"" property:
1. Only include the exception error message without any stack trace. The stack trace should never be included in the ""Message"".
2. If the content is too long please only keep the first 1000 characters.
For the ""StackTrace"" property:
1. Please assign the stack trace. 
2. If the content is too long please only keep the first 1000 characters.
The resulting JSON object should be in this format: {{ ""ExceptionType"": ""string"", ""Message"": ""string"", ""StackTrace"": ""string""}}.
Every ""string"" value must be escaped of Special Characters. Use the following rules to do so:

Example INPUT 1:
System.IO.FileNotFoundException: Could not load file or assembly 'SomeAssembly' or one of its dependencies. The system cannot find the file specified. at MyApp.Services.AssemblyLoader.Load(String assemblyPath) in C:__w\1\s\src\MyApp\Services\AssemblyLoader.cs:line 45 at MyApp.Services.PluginManager.InitializePlugins() in C:__w\1\s\src\MyApp\Services\PluginManager.cs:line 88 at MyApp.Program.Main(String[] args) in C:__w\1\s\src\MyApp\Program.cs:line 29

Example OUTPUT 1:
{{ "ExceptionType": "Could not load file or assembly 'SomeAssembly' or one of its dependencies", "Message": "Could not load file or assembly 'SomeAssembly' or one of its dependencies. The system cannot find the file specified.", "StackTrace": "at MyApp.Services.AssemblyLoader.Load(String assemblyPath) in C:\__w\1\s\src\MyApp\Services\AssemblyLoader.cs:line 45 \n at MyApp.Services.PluginManager.InitializePlugins() in C:\__w\1\s\src\MyApp\Services\PluginManager.cs:line 88 \n at MyApp.Program.Main(String[] args) in C:\__w\1\s\src\MyApp\Program.cs:line 29" }}

Example INPUT 2:
System.UnauthorizedAccessException: Access to the path 'C:\RestrictedFolder\data.txt' is denied. at System.IO.__Error.WinIOError(Int32 errorCode, String maybeFullPath) at System.IO.FileStream.Init(String path, FileMode mode, FileAccess access, Int32 rights, Boolean useRights, FileShare share, Int32 bufferSize, FileOptions options, SECURITY_ATTRIBUTES secAttrs, String msgPath) at MyApp.FileReader.ReadFile(String filePath) in C:__w\1\s\src\MyApp\FileReader.cs:line 60 at MyApp.Program.Main(String[] args) in C:__w\1\s\src\MyApp\Program.cs:line 25

Example OUTPUT 2:
{{ "ExceptionType": "Access to the path 'C:\RestrictedFolder\data.txt' is denied.", "Message": "Access to the path 'C:\RestrictedFolder\data.txt' is denied.", "StackTrace": "at System.IO.__Error.WinIOError(Int32 errorCode, String maybeFullPath) \n at System.IO.FileStream.Init(String path, FileMode mode, FileAccess access, Int32 rights, Boolean useRights, FileShare share, Int32 bufferSize, FileOptions options, SECURITY_ATTRIBUTES secAttrs, String msgPath) \n at MyApp.FileReader.ReadFile(String filePath) in C:\__w\1\s\src\MyApp\FileReader.cs:line 60 \n at MyApp.Program.Main(String[] args) in C:\__w\1\s\src\MyApp\Program.cs:line 25" }}

The ""Message"" field must exclude the stack trace, and only contain the exception message itself.
"""

USER_PROMPT_TEMPLATE = """
The given logs: ###\n{rawException}\n###
"""
prefill = "The JSON OUTPUT:"

from utils import read_file
exception = read_file("data/1.txt")
user_input = USER_PROMPT_TEMPLATE.format(rawException=exception)
# user_input = extract_exception_type_and_details_prompt(exception)

# print("--------------------------- Full prompt with variable substutions ---------------------------")
# print("USER TURN:")
# print(user_input)
response = get_completion(user_input, system_prompt=SYSTEM_PROMPT, prefill=prefill, model=model)
# # print("\n------------------------------------- LLM's response -------------------------------------")
print(response)

==================LLM gpt-4o response======================
{
    "ExceptionType": "Type 'ConfigCluster' of Role 'Cluster' raised an exception",
    "Message": "Cluster Validation failed in following tests: Name                           Value                                                                                     ----                           -----                                                                                     Network                        {Validate Network Communication}                                                          . Review the StorageClusterValidation report C:\\Windows\\temp\\StorageClusterValidationReport-2024-12-30.08-27-57 available under C:\\Windows\\Cluster\\Reports on the server for details.",
    "StackTrace": "at Trace-Error, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage.SCPSModule.3.2411.1.3\\content\\CloudDeployment\\Common\\Tracer.psm1: line 63 at Test-StorageCluster, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage

### 3. Using JSON-Mode to get a JSON output


In [ ]:
from utils import read_file
# Print LLM's response
exception = read_file("data/1.txt")
user_input = USER_PROMPT_TEMPLATE.format(rawException=exception)
response = get_completion(user_input, system_prompt=SYSTEM_PROMPT, prefill=prefill, json_mode=True, model=model)
print(response)

==================LLM gpt-4o response======================
{
    "ExceptionType": "Type 'ConfigCluster' of Role 'Cluster' raised an exception",
    "Message": "Cluster Validation failed in following tests: Name                           Value                                                                                     ----                           -----                                                                                     Network                        {Validate Network Communication} . Review the StorageClusterValidation report C:\\Windows\\temp\\StorageClusterValidationReport-2024-12-30.08-27-57 available under C:\\Windows\\Cluster\\Reports on the server for details.",
    "StackTrace": "at Trace-Error, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage.SCPSModule.3.2411.1.3\\content\\CloudDeployment\\Common\\Tracer.psm1: line 63 at Test-StorageCluster, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage.SCPSModule.3.2411.1.3\\content\\CloudDeployment\\Classes

### 4. Using Structured Output

In [ ]:

from pydantic import BaseModel

class Exception(BaseModel):
    ExceptionType: str
    Message: str
    StackTrace: str

from utils import read_file
# Print LLM's response
exception = read_file("data/1.txt")

user_input = USER_PROMPT_TEMPLATE.format(rawException=exception)

response = get_completion_with_structured_output(user_input, system_prompt=SYSTEM_PROMPT, output_format=Exception)
print(response)


==================LLM gpt-4o response======================
ExceptionType="Type 'ConfigStorage' of Role 'Storage' raised an exception" Message='Failed to configure storage on cluster Enable Storage Spaces Direct failed with error: . Run cluster validation, including the Storage Spaces Direct tests, to verify the configuration. Review the EnableClusterS2D report available under C:\\Windows\\Cluster\\Reports on the server for details.' StackTrace='at Set-SCStorageSystem, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage.SCPSModule.3.2411.1.3\\content\\JEAModules\\Microsoft.AzureStack.Fabric.Storage.3.2411.1.3\\Microsoft.AzureStack.Fabric.Storage.Deploy.psm1: line 466  at ConfigStorage, C:\\NugetStore\\Microsoft.AzureStack.Fabric.Storage.SCPSModule.3.2411.1.3\\content\\CloudDeployment\\Classes\\Storage\\Storage.psm1: line 197  at <ScriptBlock>, C:\\CloudDeployment\\ECEngine\\InvokeInterfaceInternal.psm1: line 139  at Invoke-EceInterfaceInternal, C:\\CloudDeployment\\ECEngine\\InvokeInte

### When to use Structured Outputs via function calling vs via response_format
Structured Outputs is available in two forms in the OpenAI API:

- When using function calling
- When using a json_schema response format

Function calling is useful when you are building an application that bridges the models and functionality of your application.

For example, you can give the model access to functions that query a database in order to build an AI assistant that can help users with their orders, or functions that can interact with the UI.

Conversely, Structured Outputs via response_format are more suitable when you want to indicate a structured schema for use when the model responds to the user, rather than when the model calls a tool.

Put simply:

If you are connecting the model to tools, functions, data, etc. in your system, then you should use function calling.

If you want to structure the model's output when it responds to the user, then you should use a structured response_format.


### Structured Outputs vs JSON mode
Structured Outputs is the evolution of JSON mode. While both ensure valid JSON is produced, only **Structured Outputs ensure schema adherance**. 

**Always using Structured Outputs instead of JSON mode when possible.**

However, Structured Outputs with response_format: {type: "json_schema", ...} is only supported with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots and later.

### Congrats!

You've learned how to generate structured output, you're ready to move to the next chapter. Happy prompting!